In [7]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [8]:
import joblib
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
from sklearn.metrics import mean_absolute_error

import src.config as config
from src.data_utils import transform_ts_data_info_features_and_target
from src.inference import (
    fetch_days_data,
    get_hopsworks_project,
    load_metrics_from_registry,
    load_model_from_registry,
)
from src.pipeline_utils import get_pipeline

In [9]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
import hopsworks
import numpy as np
import pandas as pd
from hsfs.feature_store import FeatureStore

import src.config as config
from src.data_utils import transform_ts_data_info_features

In [10]:
def get_feature_store() -> FeatureStore:
    project = get_hopsworks_project()
    return project.get_feature_store()

In [11]:
def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc)).tz_convert("America/New_York")
    fetch_data_from = current_date - timedelta(days=(days))
    fetch_data_to = current_date - timedelta(days=0)
    print(fetch_data_from, fetch_data_to)
    print("ghcgchcgfc")
    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_NAME, version=1)

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    print(sum(cond))
    return df[cond]
    # return df

In [12]:
print(f"Fetching data from group store ...")
ts_data = fetch_days_data(180)

Fetching data from group store ...
2024-11-11 05:15:13.672703-05:00 2025-05-10 06:15:13.672703-04:00
ghcgchcgfc
2025-05-10 06:15:13,728 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-05-10 06:15:13,799 INFO: Initializing external client
2025-05-10 06:15:13,801 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 06:15:14,861 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691


ValueError: Reading data with Hive is not supported when using hopsworks client version >= 4.0

In [ ]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-05-03 20:00:00+00:00,JC108,0
1,2025-04-14 07:00:00+00:00,JC009,0
2,2025-04-19 05:00:00+00:00,HB301,0
3,2025-05-02 08:00:00+00:00,JC075,0
4,2025-05-02 12:00:00+00:00,JC053,2
...,...,...,...
744676,2024-12-30 13:00:00+00:00,JC057,0
744677,2025-02-28 12:00:00+00:00,JC074,7
744678,2025-03-16 05:00:00+00:00,JC072,0
744681,2025-01-08 16:00:00+00:00,JC018,0
